In [1]:
import os

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from pymongo.errors import OperationFailure

load_dotenv()

llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")
embeddings = VertexAIEmbeddings(model="text-embedding-004")

MONGODB_URI = os.getenv("MONGODB_URI")
MONGODB_DATABASE = os.getenv("MONGODB_DATABASE")

prefix = "scraper_explorations"

client = MongoClient(MONGODB_URI)
collection = client[MONGODB_DATABASE][prefix]

vector_store = MongoDBAtlasVectorSearch(
    embedding=embeddings,
    collection=collection,
    index_name=prefix,
    relevance_score_fn="cosine",
)

In [2]:
try:
    vector_store.create_vector_search_index(dimensions=768)
except OperationFailure as e:
    print(f"Operation failed, does the index already exist?\n\nError: {e}")

Operation failed, does the index already exist?

Error: An index named "scraper_explorations" is already defined for collection scraper_explorations. Index names must be unique for a source collection and all its views., full error: {'ok': 0.0, 'errmsg': 'An index named "scraper_explorations" is already defined for collection scraper_explorations. Index names must be unique for a source collection and all its views.', 'code': 68, 'codeName': 'IndexAlreadyExists', '$clusterTime': {'clusterTime': Timestamp(1740842613, 1), 'signature': {'hash': b'\xb2\x016!i-)"\xaf\x93G\xb5\x84\xe0\x1d"\x15\xe4t\x97', 'keyId': 7471843386673070085}}, 'operationTime': Timestamp(1740842613, 1)}


In [3]:
client.close()